In [1]:
# Final Project
# Create a model that pridicts the winner between two Mixed Martial Arts Fighters
# dataset: https://www.reddit.com/r/datasets/comments/47a7wh/ufc_fights_and_fighter_data/

In [2]:
# use pandas to import excel spreadsheet
import pandas as pd
df = pd.read_excel('All_UFC_FIGHTS.xlsx')

In [3]:
df.head() # figure out what to keep, drop and dummy

,pageurl,eid,mid,event_name,event_org,event_date,event_place,f1pageurl,f2pageurl,f1name,f2name,f1result,f2result,f1fid,f2fid,method,method_d,ref,round,time
0,/events/UFC-1-The-Beginning-7,7,8,UFC 1 - The Beginning,Ultimate Fighting Championship,1993-11-12,"McNichols Arena, Denver, Colorado, United States",/fighter/Royce-Gracie-19,/fighter/Gerard-Gordeau-15,Royce Gracie,Gerard Gordeau,win,loss,19,15,Submission,Rear-Naked Choke,Helio Vigio,1,01:44:00
1,/events/UFC-1-The-Beginning-7,7,7,UFC 1 - The Beginning,Ultimate Fighting Championship,1993-11-12,"McNichols Arena, Denver, Colorado, United States",/fighter/Jason-DeLucia-22,/fighter/Trent-Jenkins-23,Jason DeLucia,Trent Jenkins,win,loss,22,23,Submission,Rear-Naked Choke,Joao Alberto Barreto,1,00:52:00
2,/events/UFC-1-The-Beginning-7,7,6,UFC 1 - The Beginning,Ultimate Fighting Championship,1993-11-12,"McNichols Arena, Denver, Colorado, United States",/fighter/Royce-Gracie-19,/fighter/Ken-Shamrock-4,Royce Gracie,Ken Shamrock,win,loss,19,4,Submission,Rear-Naked Choke,Helio Vigio,1,00:57:00
3,/events/UFC-1-The-Beginning-7,7,5,UFC 1 - The Beginning,Ultimate Fighting Championship,1993-11-12,"McNichols Arena, Denver, Colorado, United States",/fighter/Gerard-Gordeau-15,/fighter/Kevin-Rosier-17,Gerard Gordeau,Kevin Rosier,win,loss,15,17,TKO,Corner Stoppage,Joao Alberto Barreto,1,00:59:00
4,/events/UFC-1-The-Beginning-7,7,4,UFC 1 - The Beginning,Ultimate Fighting Championship,1993-11-12,"McNichols Arena, Denver, Colorado, United States",/fighter/Ken-Shamrock-4,/fighter/Patrick-Smith-21,Ken Shamrock,Patrick Smith,win,loss,4,21,Submission,Heel Hook,Helio Vigio,1,01:49:00


In [4]:
# keep: event_name, event_org, event_date, event_place, f1name, f2name, f1result, f2result, method, method_d, ref, round, time
# drop: f1pageurl, f2pageurl, 
# not sure about: f1fid, f2fid, f2result 
dropList = ['pageurl', 'eid', 'mid','f1pageurl', 'f2pageurl', 'f1fid', 'f2fid', 'f2result']

In [5]:
df = df.drop(dropList, axis=1)
df.columns

Index(['event_name', 'event_org', 'event_date', 'event_place', 'f1name',
       'f2name', 'f1result', 'method', 'method_d', 'ref', 'round', 'time'],
      dtype='object')

In [6]:
dummyList = ['event_name', 'event_org', 'event_place', 'f1name','f2name', 'f1result', 'method', 'method_d', 'ref', 'round']

In [7]:
#dummify categorical columns
# reference info https://www.youtube.com/watch?v=0s_1IsROgDc
df = pd.get_dummies(df, columns=dummyList)

In [8]:
columnList = df.columns

In [9]:
print(columnList)

Index(['event_date', 'time', 'event_name_UFC - The Ultimate Fighter 1 Finale',
       'event_name_UFC - The Ultimate Fighter 10 Finale',
       'event_name_UFC - The Ultimate Fighter 11 Finale',
       'event_name_UFC - The Ultimate Fighter 12 Finale',
       'event_name_UFC - The Ultimate Fighter 13 Finale',
       'event_name_UFC - The Ultimate Fighter 14 Finale',
       'event_name_UFC - The Ultimate Fighter 15 Finale',
       'event_name_UFC - The Ultimate Fighter 16 Finale',
       ...
       'ref_Vitor Ribeiro', 'ref_Wernei Cardoso', 'ref_Will Fisher',
       'ref_Wolf Menninger', 'ref_Yves Lavigne', 'round_1', 'round_2',
       'round_3', 'round_4', 'round_5'],
      dtype='object', length=3264)


In [10]:
# find column to use for labels/answers
for name in columnList:
    if name[2] == 'r':
        print(name)

f1result_NC
f1result_draw
f1result_win


In [11]:
df.shape # so big, that showing correlation doesn't seem to help much
# good candidate for tensorflow since it is now a huge dataset
# should i normalize?

(3569, 3264)

In [12]:
# Create X
X = df
X = X.drop(['f1result_win','time','event_date'], axis=1) # drop timestamp also since it blows up when creating a model
len(X.columns)

3261

In [13]:
# Create y
y = df.f1result_win

In [14]:
import numpy as np

In [15]:
# TensorFlow uses numpy arrays, so convert your pandas dataframe to a numpy array, or you'll get errors when fitting
X = np.array(X)
y = np.array(y)

In [16]:
# Your code here ...
# Here's a TensorFlow image classifier with Keras
# https://pythonprogramming.net/convolutional-neural-network-deep-learning-python-tensorflow-keras/

import tensorflow as tf
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D
#tf.enable_eager_execution() # this is necessary later when trying to pickle this model
import sklearn.metrics as metrics

In [17]:
# build the model
# reference here: https://pythonprogramming.net/introduction-deep-learning-python-tensorflow-keras/

model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(128, activation=tf.nn.relu))
model.add(tf.keras.layers.Dense(128, activation=tf.nn.relu))
model.add(tf.keras.layers.Dense(10, activation=tf.nn.softmax))

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.fit(X, y, epochs=5)

Instructions for updating:
Colocations handled automatically by placer.
Epoch 1/5
3569/3569 [==============================] - 2s 645us/sample - loss: 0.3690 - acc: 0.9695
Epoch 2/5
3569/3569 [==============================] - 2s 547us/sample - loss: 0.0212 - acc: 0.9899
Epoch 3/5
3569/3569 [==============================] - 2s 431us/sample - loss: 0.0015 - acc: 1.0000
Epoch 4/5
3569/3569 [==============================] - 2s 439us/sample - loss: 3.5509e-04 - acc: 1.0000
Epoch 5/5
3569/3569 [==============================] - 2s 442us/sample - loss: 1.7373e-04 - acc: 1.0000s - lo


In [18]:
# this worked out pretty well. It almost seems overfitted, but needs to be checked